In [ ]:
import sys
sys.path.append('/Users/HeeyunLee/Developer/cccc/functions/source')

print(sys.path)

In [ ]:
from test_configuration import plaid_client, firestore_client

import json
from datetime import date, datetime, timedelta

from plaid import ApiException
from plaid.model.transactions_get_request import TransactionsGetRequest
from plaid.model.transactions_get_response import TransactionsGetResponse

from plaid.model.accounts_balance_get_request import AccountsBalanceGetRequest
from plaid.model.accounts_get_response import AccountsGetResponse
from plaid.model.account_balance import AccountBalance

from google.cloud.firestore import DocumentReference
from google.cloud.firestore import CollectionReference

from typing import Any, Dict, Union, List

In [ ]:
def transactions_get(access_token: str, start_date: date, end_date: date):
    try:
        request = TransactionsGetRequest(
            access_token=access_token,
            start_date=start_date,
            end_date=end_date,
        )
        response: TransactionsGetResponse = plaid_client.transactions_get(
            request
        )

        return response.to_dict()

    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
def get_access_tokens(uid: str) -> Dict[str, Any]:
    user_doc = firestore_client.collection('users').document(uid)
    user_secrets_doc: DocumentReference = user_doc.collection('secrets').document(uid)
    user_secrets_snapshot = user_secrets_doc.get()

    if not user_secrets_snapshot.exists:
        return {'status': 400, 'error_message': 'access token does not exists'}
    
    access_tokens = user_secrets_snapshot.to_dict().get('plaid_access_tokens')

    return {'status': 200, 'acess_tokens': access_tokens}

In [ ]:
def update_transactions(uid: str, transactions: List[Dict[str, Any]]):
    try:
        updated_transactions_length = 0
        user_doc = firestore_client.collection('users').document(uid)
        transactions_collection: CollectionReference = user_doc.collection('transactions')

        for transaction in transactions:
            transaction_id = transaction.get('transaction_id')
            transaction_doc = transactions_collection.document(transaction_id)
            transaction_snapshot = transaction_doc.get()
            pending = transaction_snapshot.get('pending')
            
            if transaction_snapshot.exists and not pending:
                continue

            old_date: date = transaction.get('date')
            new_date = datetime(old_date.year, old_date.month, old_date.day)

            authorized_date: Union[date, None] = transaction.get('authorized_date')

            if authorized_date is not None:
                authorized_date = datetime(authorized_date.year, authorized_date.month, authorized_date.day)

            transaction.update({
                'date': new_date,
                'authorized_date': authorized_date,
            })

            if transaction_snapshot.exists:
                transaction_doc.update(transaction)
            else:
                transaction_doc.create(transaction)
            
            updated_transactions_length += 1

        return {'status': 200, 'message': 'successfully updated transactions data', 'updated_transactions_length': updated_transactions_length}
    except Exception as e:
        return {'status': 404, 'error_message': str(e)}

In [ ]:
def accounts_balance_get(access_token: str):
    try:
        request = AccountsBalanceGetRequest(access_token)
        response: AccountsGetResponse = plaid_client.accounts_balance_get(
            request)

        return response.to_dict()
    except ApiException as e:
        exceptions: dict = json.loads(e.body)
        return exceptions

In [ ]:
def update_accounts(uid: str, ins_id: str, accounts: List[dict]) -> Dict[str, Any]:
    try:
        user_doc = firestore_client.collection('users').document(uid)
        accounts_ref: CollectionReference = user_doc.collection('accounts')

        # Update Accounts in `Accounts` collections
        for account in accounts:

            # Update Accounts Collections
            account_id: str = account.get('account_id')
            print(f'accountId: {account_id}')

            account_doc_ref = accounts_ref.document(account_id)
            account_snapshot = account_doc_ref.get()

            balances: Union[AccountBalance, None] = account.get('balances')

            data = {
                'account_id': account.get('account_id'),
                'balances': {
                    'available': balances.get('available'),
                    'current': balances.get('current'),
                    'iso_currency_code': balances.get('iso_currency_code'),
                    'limit': balances.get('limit'),
                    'unofficial_currency_code': balances.get('unofficial_currency_code'),
                },
                'mask': account.get('mask'),
                'name': account.get('name'),
                'official_name': account.get('official_name'),
                'subtype': str(account.get('subtype')),
                'type': str(account.get('type')),
                'verification_status': account.get('verification_status'),
                'account_last_synced_time': datetime.now(),
                'institution_id': ins_id,
            }

            if account_snapshot.exists:
                account_doc_ref.update(data)
            else:
                account_doc_ref.set(data)

            result = {
                'status': 200,
                'message': 'successfully updated accounts data',
            }

        return result
    except Exception as e:
        result = {
            'status': 404,
            'error_message': str(e),
        }

        return result


In [ ]:
def transactions_refresh(request: Dict[str, Any]):
    # TODO: ^change the type of request to `flask.Request`

    # # TODO: uncomment below for release
    # # Parsing data from request to get `uid` and `public_token`
    # data = request.data
    # data_dict: dict = json.loads(data)
    # public_token: Union[str, None] = data.get('public_token')
    # uid: Union[str, None] = data_dict.get('uid')   

    uid: Union[str, None] = request.get('uid') # for sandbox
    now = datetime.now()
    end_date = now.date()
    start_date = (now - timedelta(60)).date()

    # Get stored access_tokens
    access_tokens_response = get_access_tokens(uid)
    access_tokens: Union[List[str], None] = access_tokens_response.get('acess_tokens')
    print(f'access_tokens: {access_tokens}')

    if access_tokens is None:
        return access_tokens_response

    for access_token in access_tokens:
        transacitons_get_response = transactions_get(
            access_token=access_token,
            start_date=start_date,
            end_date=end_date,
        )
        
        # Get Transactions data and update Firestore transactions collection
        transactions: Union[List[Dict], None] = transacitons_get_response.get('transactions')

        if transactions is None:
            error_code = transacitons_get_response.get('error_code')
            error_message = transacitons_get_response.get('error_message')
            print(f'Error code: {error_code}')
            print(f'Error message: {error_message}')

            # # TODO: uncomment below for release
            # return make_response(jsonify(status=error_code, error_message=error_message), 404)

        print(f'Fetched {len(transactions)} transaction(s)')
        update_transactions_result = update_transactions(uid, transactions)
        print(f'update_transactions_result: {update_transactions_result}')


        # Get /accounts/balance/get and update Firestore accounts collection
        balances_get_response = accounts_balance_get(access_token)
        accounts: Union[List[Dict], None] = balances_get_response.get('accounts')

        if accounts is None:
            error_code = balances_get_response.get('error_code')
            error_message = balances_get_response.get('error_message')
            print(f'Error code: {error_code}')
            print(f'Error message: {error_message}')

            # # TODO: uncomment below for release
            # return make_response(jsonify(status=error_code, error_message=error_message), 404)

        item: Union[Dict, None] = balances_get_response.get('item')
        institution_id: Union[str, None] = item.get('institution_id')
        update_account_result = update_accounts(uid, institution_id, accounts)
        print(f'update_account_result: {update_account_result}')

        # Get the status of these two firestore updates, and return
        update_transactions_status = update_transactions_result.get('status')
        update_account_result_status = update_account_result.get('status')
        print(f'''
            update_transactions_status: {update_transactions_status}
            update_account_result_status: {update_account_result_status}
        ''')

    # # TODO: uncomment below for release
    # result = jsonify(
    #     update_transactions_status=update_transactions_status,
    #     update_account_result_status=update_account_result_status,
    # )

    # return make_response(result)

In [ ]:
data = {
    'uid': 'uid',
}

transactions_refresh(data)